
- [使用 PyTorch 进行自然语言处理 (NLP)](https://www.dataquest.io/blog/natural-language-processing-nlp-with-pytorch/)

文本数据无处不在：博客、评论、聊天消息、电子邮件、支持工单、会议记录以及社交媒体帖子等等。但要大规模地理解这些数据却并非易事。与我们在数据科学中常用的整洁电子表格和数据库不同，文本数据杂乱无章、结构混乱，并且充满了人类难以理解的复杂性。

*在本教程中，我们将在 PyTorch 中演示一个能够*理解文本数据的NLP 动手任务：将推文分类为真实的或虚假的灾难报告。我们将学习如何对文本进行标记化、加载预训练模型、对其进行微调，以及在不涉及任何抽象概念或高级数学的情况下评估预测结果。

最后，您将在 PyTorch 中构建一个完整的 NLP 管道，可以确定推文是否描述的是真正的灾难。

为什么 NLP 对数据科学家如此重要
------------------

自然语言处理支持许多常见的数据科学任务：

-   **情绪分析**------从评论中理解客户意见
-   **内容分类**------自动对支持票或新闻文章进行分类
-   **聊天机器人和虚拟助手**------支持对话界面
-   **信息提取**------从文本文档中提取结构化数据

NLP 的独特之处在于**文本具有序列结构和上下文；**单词根据其位置以及与其他单词的关系构建含义。与表格数据不同，语言必须按序列进行解释，这需要专门的技术。

### 我们的灾难推文数据集

我们将使用一个真实世界的数据集：对一条推文是否描述了真实的灾难事件进行分类。该数据集非常适合学习自然语言处理 (NLP)，因为：

1.  文本简短易懂
2.  问题是二元分类（无论是否是真正的灾难）
3.  业务相关性明确（识别实际的紧急情况）
4.  结果很容易解释

例如，考虑数据集中的这些推文：

-   "加拿大萨斯喀彻温省拉龙格附近发生森林火灾"（*真实灾难*）
-   "阳光明媚，我正前往海滩#disaster #notreally"（*不是真正的灾难*）

PyTorch 中的关键 NLP 概念
-------------------

在我们进入代码之前，让我们快速回顾一下现代 NLP 系统的一些关键组件。

### 标记化

文本需要先转换成数字，然后神经网络才能处理它。**标记化**会将文本分解成多个部分（标记），并为每个部分分配一个唯一的数字 ID。如果您尝试过像 ChatGPT 这样的大型语言模型 (LLM)，那么您肯定遇到过标记化------这些模型在内部使用标记来理解和生成文本。

现代标记器不仅仅按空格进行拆分；它们使用可以处理的子词标记方法：

-   **通过将生僻词**分解成更小的片段来查找
-   利用已知子词块拼写**错误**
-   通过组合熟悉的子词来生成**新词或未知词**

例如，"预处理"一词可能会被分解成诸如`"pre"`、`"process"`和 之类的词法单元`"ing"`，每个单元都有各自的数字词法单元 ID。像 GPT-4 这样的大型语言模型使用类似的技术，将输入文本分解成词法单元，以帮助模型高效处理海量词汇。

### 嵌入和向量化

一旦我们有了词条 ID，我们就需要一种能够捕捉其含义的表示方法。**嵌入**是词条的密集向量表示，它将相似的单词在高维空间中紧密排列在一起。

为了理解嵌入，想象一个多维空间，其中每个单词都有其独特的位置。在这个嵌入空间中：

-   含义相近的单词出现在一起
-   意思相反的词相距很远
-   单词之间的关系被保留为方向

例如，在训练良好的**嵌入空间**中：

-   "国王" - "男人" + "王后" ≈ "女人"（捕捉性别关系）
-   "巴黎" - "法国" + "罗马" ≈ "意大利"（捕捉首都与国家的关系）

像"灾难"这样的词可能被表示为一个由 768 个浮点数组成的向量，而类似概念，例如"灾祸"，则在这个嵌入空间中有一个与之相邻的向量。像"龙卷风"、"地震"和"洪水"这样的词会聚集在附近的区域，而像"阳光"或"生日"这样不相关的词则会聚在一起。

![文本矢量化和嵌入](https://www.dataquest.io/wp-content/uploads/2025/04/Text_vectorization_and_embeddings.png.webp)

[这些丰富的数字表示使得模型能够比简单的独热编码](https://www.dataquest.io/blog/kaggle-getting-started/)更好地理解语义关系，在简单的独热编码中，每个单词都与其他单词有相同的差异。

### NLP 中的 Transformer

一旦将 token 转换为有意义的嵌入，下一步就是使用能够有效利用这些嵌入的架构。Transformer**是**支持现代 NLP 和大型语言模型 (LLM)（包括 GPT 模型）的主流架构。与循环神经网络等较旧的架构不同，Transformer 使用一种称为**自注意力**机制（GPT 中的"T"代表"Transformer"）来处理文本，这使得它们能够：

-   同时分析所有单词，大大加快训练和推理速度
-   捕捉单词之间的复杂关系，无论它们在句子中的距离如何
-   更有效地理解微妙的背景和细微差别

在本教程中，我们将使用一个名为**DistilBERT**的转换器模型，它是广泛使用的 BERT 模型的紧凑、高效版本，以更少的参数保留了出色的性能。

### NLP中的迁移学习

基于 Transformer 的模型的一大优势在于它们与**迁移学习**兼容，而迁移学习是现代 NLP 的核心技术。迁移学习是指采用已在海量文本数据集上进行预训练的模型（GPT 中的"P"代表"预训练"），并针对特定的 NLP 任务（例如情绪分析、问答或灾难推文分类）对其进行微调。

这种方法非常有效，因为它：

-   减少对大型特定任务数据集的需求
-   大幅缩短训练时间
-   提高性能，尤其是在较小的数据集上

例如，ChatGPT（GPT-3.5 或 GPT-4 的微调版本）利用迁移学习将通用语言模型应用于特定的对话任务。同样，在本教程中，我们将利用迁移学习和 DistilBERT 进行结合，微调其预训练权重，以将推文分类为与灾难相关或非灾难相关，从而显著简化训练过程。

### 预训练模型和 Hugging Face Hub

[Hugging Face](https://huggingface.co/)已成为 NLP 模型的核心枢纽，为数千个预训练模型提供统一的 API。他们的`transformers`库只需几行代码即可轻松加载、微调和部署最先进的模型。

我们将使用的模型包括：

-   经过预训练的 DistilBERT 库，能够理解语言模式
-   在顶部添加一个分类头，我们将针对特定任务进行训练（微调）

好的，解决了这些问题后，让我们开始构建我们的灾难推文分类器！

准备数据集
-----

首先加载必要的库和数据集。如果您使用[Google Colab](https://www.dataquest.io/blog/getting-started-with-google-colab-for-deep-learning/)进行编程，则所需的库已安装。如果您在本地运行，则可能需要先安装它们：

```
# Uncomment and run if needed
# !pip install pandas numpy matplotlib seaborn torch scikit-learn
```

现在，让我们导入必要的库并加载我们的[训练](https://dsserver-prod-resources-1.s3.amazonaws.com/nlp/train.csv)和[测试](https://dsserver-prod-resources-1.s3.amazonaws.com/nlp/test.csv)数据集。

```
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
# For reproducibility across both CPU and GPU
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# Additional seeds for CUDA operations
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  # for multi-GPU setups
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# Set Pandas display options
pd.set_option('display.max_colwidth', 100)
# Load the datasets
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# Take a first look at the data
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape:     {test_df.shape}")
train_df.head()
```

预期输出：

```
Training set shape: (7613, 5)
Test set shape:     (3263, 4)
   id keyword location                                               text  target
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...       1
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada       1
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...       1
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...       1
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...       1
```

让我们看看我们有哪些列以及它们包含什么：

```
# Check for missing values
print("Missing values per column:")
print(train_df.isnull().sum())
# Target distribution
print("\nTarget distribution:")
print(train_df['target'].value_counts())
```

预期输出：

```
Missing values per column:
id            0
keyword      61
location   2533
text          0
target        0
dtype: int64
Target distribution:
0    4342
1    3271
Name: target, dtype: int64
```

我们可以看到：

-   我们的数据集包含 7,613 条带有标记目标的推文
-   目标是二进制的（0 = 不是灾难，1 = 真正的灾难）
-   非灾难推文（4,342 条）比灾难推文（3,271 条）多
-   许多推文缺失`location`，有些缺少`keyword`信息
    -   由于我们只对`text`和感兴趣`target`，所以我们不需要担心处理丢失的数据

### 用词云探索数据

让我们使用词云来可视化文本数据，以便更好地理解灾难和非灾难推文的内容。如果您在本地运行此代码，则可能需要先安装以下必需的库才能继续：

```
# Uncomment and run if needed
# !pip install nltk wordcloud
```

现在，让我们为每个类别创建词云：

```
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
# Download stopwords if needed
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def create_wordcloud(text_series, title):
    # Combine all text
    text = ' '.join(text_series)
    # Create and generate a word cloud image
    wordcloud = WordCloud(width=800, height=400,
                          background_color='white',
                          stopwords=stop_words,
                          max_words=150,
                          collocations=False).generate(text)
    # Display the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title)
    plt.show()
# Create word clouds for each class
create_wordcloud(train_df[train_df['target'] == 1]['text'],
                 'Words in Disaster Tweets')
print()
create_wordcloud(train_df[train_df['target'] == 0]['text'],
                 'Words in Non-Disaster Tweets')
```

![灾难推文词云](https://www.dataquest.io/wp-content/uploads/2025/04/Disaster_tweet_WordCloud.jpg.webp)

![非灾难推文词云](https://www.dataquest.io/wp-content/uploads/2025/04/Non_disaster_tweet_WordCloud.jpg.webp)

观察这些词云，我们可以注意到几个有趣的模式：

1.  **灾难推文**（上图）突出地出现了"灾难"、"火灾"、"警察"、"死亡"、"洪水"、"紧急情况"和"炸弹"等词语------这些词语显然与灾难性事件有关。
2.  **非灾难推文**（下图）包含更多日常用语，如"喜欢"、"得到"、"新的"、"日子"、"时间"、"去"和"人们"。
3.  **编码问题**：两个词云都显示"Û"字符，这是由于特殊字符未正确解码而发生的编码错误。这表明我们需要在预处理过程中处理字符编码问题。
4.  **URL 和 Twitter 特定内容**：两个云都显示"http"和"co"（来自" <http://t.co/> ..."链接），这表明我们需要在预处理期间删除 URL。
5.  **RT 和 amp**：我们发现 Twitter 特有的术语，如"RT"（转发）和"amp"（来自 HTML 编码的"&"符号），对于分类没有任何价值。

这些见解将为我们的文本预处理策略提供参考。

### 基本文本预处理

根据我们的词云分析，我们需要在将文本数据输入模型之前对其进行清理。尽管现代 Transformer 非常稳健，但我们仍应解决以下几个问题：

```
def clean_text(text):
    """Basic text cleaning function"""
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remove Twitter-specific content
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)     # Remove hashtag symbols but keep the words
    text = re.sub(r'rt\s+', '', text)  # Remove RT (retweet) indicators
    # Remove HTML entities (like &)
    text = re.sub(r'&\w+;', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Handle encoding errors like 'Û'
    text = re.sub(r'Û', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text
# Apply cleaning to the text column
train_df['cleaned_text'] = train_df['text'].apply(clean_text)
test_df['cleaned_text'] = test_df['text'].apply(clean_text)
# Display a few examples of cleaned text
print("Original vs Cleaned:")
for i in range(3):
    print(f"Original: {train_df['text'].iloc[i]}")
    print(f"Cleaned: {train_df['cleaned_text'].iloc[i]}")
    print()
```

预期输出：

```
Original vs Cleaned:
Original: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Cleaned: our deeds are the reason of this earthquake may allah forgive us all
Original: Forest fire near La Ronge Sask. Canada
Cleaned: forest fire near la ronge sask canada
Original: All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
Cleaned: all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected
```

这种预处理解决了我们在词云中发现的问题：

1.  删除 URL（包括"http"和"t.co"链接）
2.  消除 Twitter 特定内容，例如提及和 RT 指标
3.  处理诸如"Û"字符之类的编码问题
4.  删除特殊字符、数字和多余的空格
5.  将所有文本转换为小写以保持一致性

将文本转换为张量
--------

现在，我们将文本数据转换为 PyTorch 可以处理的格式。为此，我们将使用 Hugging Face`transformers`库，它已成为 NLP 中使用 Transformer 模型的标准工具包。

如果您在本地运行此代码，则需要安装`transformers`库：

```
# Uncomment and run if needed
# !pip install transformers
```

现在让我们设置我们的标记器：

```
from transformers import DistilBertTokenizer
# Load pretrained tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# Example of tokenization
example_text = "PyTorch is great for NLP"
tokens = tokenizer.tokenize(example_text)
token_ids = tokenizer.encode(example_text)
print(f"Original text: {example_text}")
print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")
```

预期输出：

```
Original text: PyTorch is great for NLP
Tokens: ['p', '##yt', '##or', '##ch', 'is', 'great', 'for', 'nl', '##p']
Token IDs: [101, 1052, 22123, 2953, 2818, 2003, 2307, 2005, 17953, 2361, 102]
```

让我们创建一个函数来使用上面的方法标记我们的文本数据`DistilBertTokenizer`：

```
def tokenize_text(texts, tokenizer, max_length=128):
    """
    Tokenize a list of texts using the provided tokenizer
    Returns input IDs and attention masks
    """
    # Tokenize all texts at once
    encodings = tokenizer(
        list(texts),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return encodings['input_ids'], encodings['attention_mask']
```

### 理解注意力面具

当我们使用上述函数对文本进行标记时，我们得到两个重要的输出：

1.  **输入 ID**：这些是我们的标记的数字表示。每个单词或子单词都会根据标记器词汇表转换为唯一的数字。
2.  **注意力掩码**：这些是二进制张量（仅包含 0 和 1），它们告诉模型哪些标记需要"注意"以及哪些标记需要忽略。

为什么我们*需要*注意力掩码？因为我们是批量处理推文的，而推文的长度各不相同。为了使所有序列的长度保持一致，我们会在较短的序列中添加**填充标记**。注意力掩码中，真实标记用 1 表示，填充标记用 0 表示，这告诉模型："关注*真实内容*，忽略填充。"

例如，如果我们有一条推文，其长度为 10 个标记，但我们要填充到 128 个标记，则注意力掩码将如下所示：

```
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ..., 0]
```

这种机制对于 Transformer 模型至关重要，因为它可以确保它们不会尝试从人工填充标记中提取意义。

创建数据集和数据加载器
-----------

接下来，我们需要将数据分成训练集和验证集，并准备进行训练。这涉及几个步骤：

1.  **拆分数据**：我们将数据集分为训练数据（模型学习的数据）和验证数据（我们用来评估模型性能的数据）
2.  **对文本进行标记**：我们将所有文本转换为标记 ID 和注意掩码
3.  **创建 DataLoaders**：我们设置高效的管道，在训练期间将数据提供给我们的模型

这一过程值得我们关注，因为它：

-   通过提供单独的评估数据帮助防止过度拟合
-   支持批处理，加快训练速度
-   标准化模型的输入形状

让我们实现以下步骤：

```
# Split data into train and validation sets
train_texts, val_texts, train_targets, val_targets = train_test_split(
    train_df['cleaned_text'].values,
    train_df['target'].values,
    test_size=0.1,
    random_state=42,
    stratify=train_df['target']  # Maintain class distribution
)
print(f"Training texts: {len(train_texts)}")
print(f"Validation texts: {len(val_texts)}")
# Set the batch size for effecient training
batch_size = 16
# Process training data
train_input_ids, train_attention_masks = tokenize_text(train_texts, tokenizer)
train_targets = torch.tensor(train_targets, dtype=torch.long)
# Process validation data
val_input_ids, val_attention_masks = tokenize_text(val_texts, tokenizer)
val_targets = torch.tensor(val_targets, dtype=torch.long)
# Create tensor datasets
train_dataset = TensorDataset(
    train_input_ids,
    train_attention_masks,
    train_targets
)
val_dataset = TensorDataset(
    val_input_ids,
    val_attention_masks,
    val_targets
)
# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size
)
# Look at a single batch
batch = next(iter(train_loader))
input_ids, attention_mask, targets = batch
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention mask shape: {attention_mask.shape}")
print(f"Targets shape: {targets.shape}")
```

预期输出：

```
Training texts: 6851
Validation texts: 762
Input IDs shape: torch.Size([16, 128])
Attention mask shape: torch.Size([16, 128])
Targets shape: torch.Size([16])
```

这个输出告诉我们：

-   我们有 6,851 条推文用于训练，762 条推文用于验证
-   每批包含 16 条推文（我们选择的批次大小）
-   每条推文由 128 个标记（包括填充）表示
-   注意力掩码与输入形状相匹配，1 表示真实标记，0 表示填充
-   我们的目标只是每条推文 0 或 1（非灾难或灾难）

微调预训练的 Transformer
------------------

现在，我们准备加载一个预训练的 Transformer 模型，并对其进行微调，以完成灾难推文分类任务。选择 Transformer 模型时，应考虑模型大小、推理速度、任务复杂度以及可用的计算资源等因素。对于推文分类（一项相对简单的短文本任务）而言，像 DistilBERT 这样规模较小、效率更高的 Transformer 是理想之选，因为它在速度和准确率之间取得了平衡，且无需耗费大量资源。

让我们首先建立模型：

```
from transformers import DistilBertForSequenceClassification
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# Load pretrained model
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2  # Binary classification
)
# Move model to device
model = model.to(device)
# Set up optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
```

预期输出：

```
Using device: cuda
model.safetensors: 100%
 268M/268M [00:01<00:00, 257MB/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
```

这个输出告诉我们一些重要的事情：

1.  我们正在使用 GPU 进行训练（感谢 Google Colab！）
2.  模型加载成功（268MB参数）
3.  一些模型权重是新初始化的------特别是我们在基础模型之上添加的分类层
4.  该模型需要进行训练才能用于预测（这正是我们要做的！）

### 了解 F1 分数

在开始训练之前，让我们先了解如何评估我们的模型。我们将使用**F1 分数**，它是分类任务中常用的指标，尤其是在类别不平衡的情况下。

F1 分数是精确度和召回率的调和平均值：

-   **准确度**：在我们预测为灾难的所有推文中，有多少条实际上是灾难？
-   **回想一下**：在所有实际的灾难推文中，我们正确识别了多少条？

F1 = 2 *（精确度*/召回率）/（精确度+召回率）

我们使用 F1 而不是准确度，因为：

1.  它平衡了准确率和召回率，这对于灾害检测很重要
2.  它适用于不平衡数据集（其中一个类比另一个类出现得更多）
3.  它对假阳性和假阴性都会进行惩罚

### 训练模型

现在，让我们实现训练循环。训练过程在 GPU 上通常需要 3-5 分钟，在 CPU 上则需要几个小时：

```
# Training function
def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    for batch in data_loader:
        # Unpack and move batch to device
        input_ids, attention_mask, targets = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        # Forward pass
        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=targets
        )
        loss = outputs.loss
        logits = outputs.logits
        # Backward pass
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        # Calculate accuracy
        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == targets)
        total_predictions += len(targets)
    # Calculate average loss and accuracy
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / total_predictions
    return avg_loss, accuracy
# Evaluation function
def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_targets = []
    all_preds = []
    with torch.no_grad():
        for batch in data_loader:
            # Unpack and move batch to device
            input_ids, attention_mask, targets = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            targets = targets.to(device)
            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=targets
            )
            loss = outputs.loss
            logits = outputs.logits
            total_loss += loss.item()
            # Calculate accuracy
            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == targets)
            total_predictions += len(targets)
            # Store targets and predictions for F1 score
            all_targets.extend(targets.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    # Calculate average loss and accuracy
    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions.double() / total_predictions
    f1 = f1_score(all_targets, all_preds)
    return avg_loss, accuracy, f1
# Training loop
epochs = 3
best_f1 = 0
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    # Evaluate
    val_loss, val_acc, val_f1 = evaluate(model, val_loader, device)
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}, Val F1: {val_f1:.4f}")
    if val_f1 > best_f1:
        best_f1 = val_f1
        # In a real scenario, we'd save the model here
        # torch.save(model.state_dict(), "best_model.pt")
    print()
```

预期输出：

```
Epoch 1/3
Train Loss: 0.4351, Train Accuracy: 0.8097
Val Loss: 0.3737, Val Accuracy: 0.8491, Val F1: 0.8130
Epoch 2/3
Train Loss: 0.3300, Train Accuracy: 0.8678
Val Loss: 0.3899, Val Accuracy: 0.8399, Val F1: 0.8045
Epoch 3/3
Train Loss: 0.2410, Train Accuracy: 0.9085
Val Loss: 0.4127, Val Accuracy: 0.8451, Val F1: 0.8072
```

查看这些结果，我们可以看到：

1.  **训练损失和准确率**：正如预期，随着模型的学习，训练指标稳步提升。到第 3 个 epoch 时，该模型在训练数据上的准确率已接近 91%。
2.  **验证性能**：验证指标反映的情况更为复杂。虽然准确率保持相对稳定（约为 84-85%），但我们发现 F1 分数有所波动，最佳值通常出现在第一个 epoch 之后。
3.  **潜在的过拟合**：训练集和验证集指标（尤其是损失）之间的差距越来越大，表明模型开始对训练数据过拟合。验证集 F1 在 epoch 1 之后略有下降，这表明我们可能需要在生产场景中考虑应用一些[正则化。](https://www.dataquest.io/blog/regularization-in-machine-learning/)

评估结果
----

现在我们已经训练了我们的模型，让我们在验证集上更彻底地评估它，以了解它的优势和局限性。

```
# Detailed evaluation
model.eval()
all_targets = []
all_preds = []
all_probs = []  # For prediction probabilities
with torch.no_grad():
    for batch in val_loader:
        # Unpack and move batch to device
        input_ids, attention_mask, targets = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        targets = targets.to(device)
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        # Get predictions
        _, preds = torch.max(logits, dim=1)
        # Store targets, predictions, and probabilities
        all_targets.extend(targets.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())  # For positive class
# Classification report
print(classification_report(all_targets, all_preds, target_names=['Not Disaster', 'Disaster']))
# Confusion matrix
cm = pd.crosstab(
    pd.Series(all_targets, name='Actual'),
    pd.Series(all_preds, name='Predicted')
)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.show()
```

预期输出：

```
              precision    recall  f1-score   support
Not Disaster       0.83      0.91      0.87       435
    Disaster       0.87      0.76      0.81       327
    accuracy                           0.85       762
   macro avg       0.85      0.83      0.84       762
weighted avg       0.85      0.85      0.84       762
```

分类报告和混淆矩阵提供了有价值的见解：

1.  **总体准确度**：我们的模型在验证集上达到了 85% 的准确度，这对于只需极少预处理的复杂 NLP 任务来说相当不错。
2.  **类别表现**：该模型在"非灾难"推文（F1 得分为 0.87）上的表现略好于"灾难"推文（F1 得分为 0.81）。

![混淆矩阵](https://www.dataquest.io/wp-content/uploads/2025/04/Confusion_matrix.png.webp)

1.  **混淆矩阵分析**：查看混淆矩阵，我们可以看到：
    -   397 条真阴性（正确识别的非灾难推文）
    -   247 条真实阳性（正确识别的灾难推文）
    -   38 条误报（非灾难推文被错误地标记为灾难）
    -   80 条假阴性（错过灾难推文）
2.  **错误类型影响**：该模型漏报真实灾害（80 个误报）的可能性略高于发出误报（38 个误报）。在现实世界的灾害监测系统中，可能需要根据漏报真实紧急情况与调查误报的相对成本来调整这种权衡。

### 根据新数据进行预测

最后，让我们使用我们的模型对测试集进行预测：

```
# Process test data
test_input_ids, test_attention_masks = tokenize_text(test_df['cleaned_text'].values, tokenizer)
# Create test dataloader
test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
# Generate predictions
model.eval()
test_preds = []
test_probs = []
with torch.no_grad():
    for batch in test_loader:
        # Unpack and move batch to device
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        # Forward pass
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        # Get predictions
        _, preds = torch.max(logits, dim=1)
        # Store predictions and probabilities
        test_preds.extend(preds.cpu().numpy())
        test_probs.extend(probs[:, 1].cpu().numpy())
# Add predictions to test dataframe
test_df['predicted_target'] = test_preds
test_df['disaster_probability'] = test_probs
# Display a sample of predictions
print("Sample predictions on the test set:")
sample_results = test_df[['text', 'predicted_target', 'disaster_probability']].sample(10)
sample_results
```

预期输出：

```
Sample predictions on the test set:
                                                                                                    text    predicted_target    disaster_probability
149  #MustRead: Vladimir #Putin Issues Major Warning But Is It Too Late To Escape Armageddon? by @PCr...                   1                0.820633
2028    tarmineta3: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a...                0                0.079320
2559                                      @MeganRestivo I am literally screaming for you!! Congratulations!                0                0.011177
800                                                          @PahandaBear @Nethaera Yup EU crashed too :P                  0                0.039989
1237    Angry Woman Openly Accuses NEMA Of Stealing Relief Materials Meant For IDPs: An angry Internally...                0                0.078757
2448    Photo: theonion: Rescuers Heroically Help Beached Garbage Back Into OceanåÊ http://t.co/YcSmt7ovoc                 1                0.739298
1566    ...@jeremycorbyn must be willing to fight and 2 call a spade a spade. Other wise very savvy piec...                0                0.032905
691              Emergency services called to Bacup after 'strong' chemical smells http://t.co/hJJ7EFTJ7O                1                0.934740
3103    T Shirts $10 male or female get wit me. 2 days until its game changin time. War Zone single will...                0                0.015829
2325    #Np love police @PhilCollinsFeed On #LateNiteMix Uganda Broadcasting Corporation. UBC 98FM #Radi...                0                0.340198

```

结果
--

观察上面的测试预测，我们可以看到，该模型**对带有明显紧急相关语言的推文**（例如涉及化学气味或紧急警告的推文）分配较高的灾难概率。相反，它对**明显非灾难性的推文**（包括个人庆祝活动、随意对话和促销内容）分配的灾难概率非常低。

一个有趣的案例是*《洋葱报》*上一条关于救援人员和垃圾的讽刺推文。虽然模型将其预测为灾难（0.74），但这突显了它**受到与灾难相关的文字词汇的强烈影响**，即使在幽默语境中使用也是如此。另一方面，一条提到*"警察"*一词的推文得分中等（约0.34），但仍然被正确分类为非灾难。这表明模型已经学会了某些关键词既可以出现在灾难场景中，也可以出现在非灾难场景中。

总体而言，这些预测反映了一种学会平衡文字线索和更广泛背景的模型，即使没有大量的预处理或长期的训练也能表现出稳定的性能。

后续步骤和资源
-------

准备好进一步提升你的 NLP 技能了吗？以下是一些自然而然的后续步骤：

### 尝试其他任务

-   **多类分类**：尝试按主题对新闻文章进行分类
-   **情绪分析**：将电影评论分为正面或负面
-   **命名实体识别**：从文本中提取人物、地点和组织

### 探索更多高级技术

-   尝试其他 Transformer 模型（BERT、RoBERTa、T5）
-   文本数据增强实验
-   实施注意力可视化来解释模型决策
-   添加早期停止以防止过度拟合（正如我们在训练结果中看到的那样）
-   应用交叉验证进行更稳健的评估
-   探索超参数调整以优化学习率、批量大小等。

### 推荐资源

-   [拥抱脸部文档](https://huggingface.co/docs)
-   [PyTorch NLP 教程](https://pytorch.org/tutorials/)
-   [PyTorch 入门](https://www.dataquest.io/blog/pytorch-for-deep-learning/)
-   [PyTorch 中的序列模型](https://www.dataquest.io/blog/sequence-models-in-pytorch/)

文本数据可能看起来杂乱复杂，但像 PyTorch 和预训练的 Transformer 这样的现代工具却让它变得出奇地容易上手。从小处着手，循序渐进，往往是提升 NLP 技能和信心的最佳途径。

关键术语回顾
------

-   **标记化**：将文本转换为标记（单词、子单词或字符）的过程
-   **嵌入**：捕捉语义关系的标记的密集向量表示
-   **迁移学习**：将预训练模型中的知识应用于新任务
-   **Transformer**：一种利用自注意力机制处理序列的神经网络架构
-   **微调**：通过更新参数使预训练模型适应特定任务
-   **注意力机制**：一种允许模型关注输入相关部分的机制
-   **注意掩码**：二进制张量，告诉模型哪些标记是真实内容，哪些是填充
-   **F1 分数**：平衡精度和召回率的指标，适用于不平衡数据集
-   **Hugging Face**：为 NLP 提供工具和预训练模型的组织